In [9]:
import sqlite3
import pandas as pd
from IPython.display import display, Markdown

# --- CONFIGURACIÓN ---
# Ruta a tu base de datos
DB_PATH = 'data/database/parlamento.db'
# Número de filas a mostrar por tabla
LIMIT = 5

def display_database_overview(db_path, limit):
    """
    Se conecta a la BD y muestra las primeras N filas de cada tabla
    de manera ordenada en un notebook.
    """
    try:
        with sqlite3.connect(db_path) as conn:
            cursor = conn.cursor()
            
            # Obtener la lista de todas las tablas
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
            tables = [table[0] for table in cursor.fetchall()]
            
            display(Markdown(f"# 📊 Resumen de la Base de Datos: `{db_path}`"))
            display(Markdown(f"Mostrando las primeras **{limit} filas** de cada tabla."))
            
            for table in tables:
                if table.startswith('sqlite_'):
                    continue
                
                try:
                    # Usar pandas para leer la consulta SQL y obtener un DataFrame
                    df = pd.read_sql_query(f"SELECT * FROM {table} LIMIT {limit}", conn)
                    
                    # Mostrar el nombre de la tabla como un título
                    display(Markdown(f"--- \n### 🏛️ Tabla: `{table}`"))
                    
                    if df.empty:
                        display(Markdown("_Esta tabla está actualmente vacía._"))
                    else:
                        # La función display() renderiza el DataFrame como una tabla HTML
                        display(df)
                        
                except Exception as e:
                    display(Markdown(f"No se pudo leer la tabla `{table}`. Error: {e}"))

    except sqlite3.Error as e:
        print(f"❌ Error de base de datos: {e}")

# --- Ejecutar la función para mostrar el resumen ---
display_database_overview(DB_PATH, LIMIT)

# 📊 Resumen de la Base de Datos: `data/database/parlamento.db`

Mostrando las primeras **5 filas** de cada tabla.

--- 
### 🏛️ Tabla: `dim_parlamentario`

,mp_uid,nombre_completo,nombre_propio,apellido_paterno,apellido_materno,genero,fecha_nacimiento,lugar_nacimiento,diputadoid,senadorid,bcn_person_id,bcn_uri,url_foto,twitter_handle,sitio_web_personal,profesion,url_historia_politica,fecha_extraccion
0,1,Carlos Bombal Otaegui,Carlos,Bombal,Otaegui,Masculino,None,None,415,None,2721,http://datos.bcn.cl/recurso/persona/2721,https://www.bcn.cl/laborparlamentaria/imagen/2...,None,None,Abogado,https://www.bcn.cl/historiapolitica/resenas_pa...,2025-09-06
1,2,Ramón Arias,Ramón,Arias,None,Masculino,None,None,None,None,2483,http://datos.bcn.cl/recurso/persona/2483,https://www.bcn.cl/laborparlamentaria/imagen/2...,None,None,None,https://www.bcn.cl/historiapolitica/resenas_pa...,2025-09-06
2,3,José Plácido Musalem Saffie,José Plácido,Musalem,Saffie,Masculino,None,None,None,None,2315,http://datos.bcn.cl/recurso/persona/2315,https://www.bcn.cl/laborparlamentaria/imagen/2...,None,None,Abogado,https://www.bcn.cl/historiapolitica/resenas_pa...,2025-09-06
3,4,Arístides Benavente Carvajal,Arístides,Benavente,Carvajal,Masculino,None,None,None,None,458,http://datos.bcn.cl/recurso/persona/458,https://www.bcn.cl/laborparlamentaria/imagen/4...,None,None,None,https://www.bcn.cl/historiapolitica/resenas_pa...,2025-09-06
4,5,Juan de Dios Peñafiel Varela,Juan de Dios,Peñafiel,Varela,Masculino,None,None,None,None,597,http://datos.bcn.cl/recurso/persona/597,https://www.bcn.cl/laborparlamentaria/imagen/5...,None,None,None,https://www.bcn.cl/historiapolitica/resenas_pa...,2025-09-06


--- 
### 🏛️ Tabla: `dim_periodo_legislativo`

...

,source_id,entity_id,entity_type,source_name,url,last_checked_at
0,10,17450-11,bill,senado_boletin,https://tramitacion.senado.cl/wspublico/tramit...,2025-09-07 21:01:13
1,11,17450-11,bill,camara_boletin,https://opendata.camara.cl/camaradiputados/WSe...,2025-09-07 21:01:13
2,12,17450-11,bill,bcn_proyecto,https://datos.bcn.cl/recurso/cl/proyecto-de-le...,2025-09-07 21:01:13
3,13,2439-20,bill,senado_boletin,https://tramitacion.senado.cl/wspublico/tramit...,2025-09-07 21:01:13
4,14,2439-20,bill,camara_boletin,https://opendata.camara.cl/camaradiputados/WSe...,2025-09-07 21:01:13


--- 
### 🏛️ Tabla: `speech_turns`

_Esta tabla está actualmente vacía._

--- 
### 🏛️ Tabla: `interactions`

_Esta tabla está actualmente vacía._

In [10]:
import sqlite3
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, Markdown

# --- CONFIGURACIÓN ---
# Ajusta la ruta a tu base de datos si es necesario
DB_PATH = 'data/database/parlamento.db'

# Opciones para que pandas muestre todas las columnas
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 50)

In [11]:
# Este widget crea un cuadro de texto para que ingreses el boletín fácilmente
bill_id_widget = widgets.Text(
    value='2439-20',
    placeholder='Ingresa un Boletín (ej: 17450-11)',
    description='Boletín:',
    disabled=False
)

display(Markdown("### 📜 Ingresa el número de boletín que quieres analizar:"))
display(bill_id_widget)

### 📜 Ingresa el número de boletín que quieres analizar:

Text(value='2439-20', description='Boletín:', placeholder='Ingresa un Boletín (ej: 17450-11)')

In [12]:
# Establecemos la conexión una sola vez
try:
    conn = sqlite3.connect(DB_PATH)
    print("✅ Conexión a la base de datos exitosa.")
except sqlite3.Error as e:
    print(f"❌ Error de base de datos: {e}")

✅ Conexión a la base de datos exitosa.


In [13]:
# Usamos el valor del widget en nuestra consulta
bill_id = bill_id_widget.value

display(Markdown(f"# 📖 Vida y Trayectoria del Proyecto de Ley: {bill_id}"))
display(Markdown("--- \n## ℹ️ Ficha Principal"))

df_bill = pd.read_sql_query("SELECT * FROM bills WHERE bill_id = ?", conn, params=(bill_id,))

if df_bill.empty:
    display(Markdown(f"**Error:** No se encontró ningún proyecto con el boletín `{bill_id}`."))
else:
    # Transponemos el dataframe para una mejor visualización de la ficha
    display(df_bill.set_index('bill_id').T)

# 📖 Vida y Trayectoria del Proyecto de Ley: 2439-20

--- 
## ℹ️ Ficha Principal

bill_id,2439-20
titulo,"Sustituye ley N° 19.366, que sanciona el tráfi..."
resumen,None
tipo_proyecto,None
fecha_ingreso,1999-12-14
etapa,Tramitación terminada
subetapa,/
iniciativa,Mensaje
origen,C.Diputados
urgencia,Sin urgencia
resultado_final,Publicado


In [14]:
display(Markdown("--- \n## ✍️ Autores y Materias"))
bill_id = bill_id_widget.value

# NUEVA CONSULTA: Ahora obtiene el partido del parlamentario en la fecha de ingreso del proyecto
authors_query = """
    SELECT
        p.nombre_completo,
        part.sigla AS partido_en_fecha
    FROM bills b
    JOIN bill_authors ba ON b.bill_id = ba.bill_id
    JOIN dim_parlamentario p ON ba.mp_uid = p.mp_uid
    LEFT JOIN parlamentario_mandatos m ON p.mp_uid = m.mp_uid AND b.fecha_ingreso BETWEEN m.fecha_inicio AND DATE(m.fecha_fin, '+1 day')
    LEFT JOIN dim_partidos part ON m.partido_id_mandato = part.partido_id
    WHERE b.bill_id = ?;
"""
df_authors = pd.read_sql_query(authors_query, conn, params=(bill_id,))

materias_query = """
    SELECT m.nombre
    FROM bill_materias bm
    JOIN dim_materias m ON bm.materia_id = m.materia_id
    WHERE bm.bill_id = ?
"""
df_materias = pd.read_sql_query(materias_query, conn, params=(bill_id,))

display(Markdown("### Autores"))
if df_authors.empty:
    display(Markdown("_No se encontraron autores parlamentarios (puede ser una iniciativa del Ejecutivo)._"))
else:
    display(df_authors)

display(Markdown("### Materias Asociadas"))
if df_materias.empty:
    display(Markdown("_No se encontraron materias específicas._"))
else:
    display(df_materias)

--- 
## ✍️ Autores y Materias

### Autores

_No se encontraron autores parlamentarios (puede ser una iniciativa del Ejecutivo)._

### Materias Asociadas

,nombre
0,Tráfico de drogas


In [15]:
display(Markdown("--- \n## 🛤️ Histórico de Tramitación"))
bill_id = bill_id_widget.value

tramites_query = """
    SELECT fecha_tramite, descripcion, etapa_especifica, camara
    FROM bill_tramites
    WHERE bill_id = ?
    ORDER BY fecha_tramite ASC
"""
df_tramites = pd.read_sql_query(tramites_query, conn, params=(bill_id,))

if df_tramites.empty:
    display(Markdown("_No se encontró historial de trámites._"))
else:
    display(df_tramites)

--- 
## 🛤️ Histórico de Tramitación

,fecha_tramite,descripcion,etapa_especifica,camara
0,1999-12-14,Ingreso de proyecto .,Primer trámite constitucional,C.Diputados
1,1999-12-14,Cuenta de proyecto . Pasa a Comisión Especial ...,Primer trámite constitucional,C.Diputados
2,1999-12-14,Oficio Nº 2.662 a Corte Suprema. Remite proyec...,Primer trámite constitucional,C.Diputados
3,2000-05-10,"S.E. el Pdte. de la Rep. remite indicaciones, ...",Primer trámite constitucional,C.Diputados
4,2000-06-07,"S.E. el Pdte. de la Rep. remite indicaciones, ...",Primer trámite constitucional,C.Diputados
5,2000-06-08,Oficio Nº 2878 a la Corte Suprema. Pone en con...,Primer trámite constitucional,C.Diputados
6,2000-07-11,Oficio Nº 2956 a la Corte Suprema. Pone en con...,Primer trámite constitucional,C.Diputados
7,2000-07-12,Cuenta oficio 001198 de la Corte Suprema,Primer trámite constitucional,C.Diputados
8,2000-07-17,"S.E. el Pdte. de la Rep. remite indicaciones, ...",Primer trámite constitucional,C.Diputados
9,2000-08-29,Cuenta del Mensaje 167-342 que hace presente l...,Primer trámite constitucional,C.Diputados


In [16]:
display(Markdown("--- \n## 🔗 Fuentes de Datos (URLs)"))
bill_id = bill_id_widget.value

sources_query = """
    SELECT source_name, url
    FROM entity_sources
    WHERE entity_id = ? AND entity_type = 'bill'
"""
df_sources = pd.read_sql_query(sources_query, conn, params=(bill_id,))

if df_sources.empty:
    display(Markdown("_No se encontraron URLs de origen para este boletín._"))
else:
    display(df_sources)

--- 
## 🔗 Fuentes de Datos (URLs)

,source_name,url
0,senado_boletin,https://tramitacion.senado.cl/wspublico/tramit...
1,camara_boletin,https://opendata.camara.cl/camaradiputados/WSe...
2,bcn_proyecto,https://datos.bcn.cl/recurso/cl/proyecto-de-le...
